# Jupyter Notebook implementation

First read the [README](README.md) file if you're new.

This is an example of recreatting the code in [`README`](README.md) in a Jupyter Notebook.

## Models

- 

In [1]:
import train_classification
import test_classification

import train_partseg
import test_partseg

import train_semseg
import test_semseg

import argparse

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
!nvidia-smi

Wed Aug 14 11:36:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.70                 Driver Version: 560.70         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P8              8W /  220W |    2114MiB /  12282MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Classification

- ModelNet40 dataset


### Training

The following is the same as:

```shell
python train_classification.py --model pointnet2_cls_ssg --log_dir pointnet2_cls_ssg
```

In [5]:
# user defined arguments for command line as a dictionary
args = {
    'model'   : 'pointnet2_cls_ssg',
    'log_dir' : 'pointnet2_cls_ssg'
}
classification_train_args = train_classification.CommandLineArgs(**args)
train_classification.main(classification_train_args)

PARAMETER ...
CommandLineArgs(model='pointnet2_cls_ssg', log_dir='pointnet2_cls_ssg', use_cpu=False, gpu='0', batch_size=25, num_category=40, epoch=200, learning_rate=0.001, num_point=1024, optimizer='adam', decay_rate=0.0001, use_normals=False, process_data=False, use_uniform_sample=False)
Load dataset ...
The size of train data is 9843
The size of test data is 2468
No existing model, starting training from scratch...
Epoch 1 (1/200):


 42%|████▏     | 164/393 [00:36<00:50,  4.53it/s]


KeyboardInterrupt: 

### Testing

The following is the same as:

```shell
python test_classification.py --log_dir pointnet2_cls_ssg
```

In [ ]:
args = {
    "log_dir": "pointnet2_cls_ssg",
}
classification_test_args = test_classification.CommandLineArgs(**args)
test_classification.main(classification_test_args)

## Segmentation